In [86]:
import os
import shutil
import re
import pandas as pd

In [87]:
# Define source and destination directories
src_dir = "/home/thomas/RatAnalysis/RatClustering/20240612_Footshock_Session1 copy"
dst_dir = "/home/thomas/RatAnalysis/RatClustering/Footshock_Session1_Fixed"

# Ensure the destination directory exists
os.makedirs(dst_dir, exist_ok=True)

# Loop through all files in the source directory
for filename in os.listdir(src_dir):
    if "filtered" in filename:
        continue

    if 'Ai213' in filename and ('.CSV' in filename or '.csv' in filename):

        start_index = filename.find('Ai213')
        # Remove all characters before 'Ai213' and change .CSV to .csv
        new_filename = filename[start_index:].replace('.CSV', '.csv')

        # Extract the grouping pattern Ai213_x=x_#x using regex
        match = re.search(r'Ai213_\d+-\d+_#\d+', new_filename)
        if match:
            if "Side_viewDLC_Resnet50" in filename:
                new_filename = match.group(0) + "_Pose_Data.csv"
            
            # Create a subdirectory based on the matched pattern
            subfolder_name = match.group(0)
            subfolder_path = os.path.join(dst_dir, subfolder_name)
            os.makedirs(subfolder_path, exist_ok=True)

            # Construct full file paths
            old_filepath = os.path.join(src_dir, filename)
            new_filepath = os.path.join(subfolder_path, new_filename)

            # Move and rename the file
            shutil.copy(old_filepath, new_filepath)

print("Files have been renamed, grouped into folders, and moved to the new directory.")


Files have been renamed, grouped into folders, and moved to the new directory.


In [88]:
def fix_void_timing(df_path):
    df = pd.read_csv(df_path)
    df['seconds'] = df['Var4'].str.extract(r'(\d+\.\d+)')
    df = df[['seconds']]
    df.to_csv(df_path)

In [89]:
def fix_pose_data(df_path):
    df = pd.read_csv(df_path)

    body_parts = df.iloc[0, 1:] 
    coords = df.iloc[1, 1:] 

    # Initialize an empty list to hold the new column names
    new_columns = []
    new_columns.append(f'Image')
    # Create new column names based on body parts and coordinates
    for part, coord in zip(body_parts, coords):
        new_columns.append(f'{part}_{coord}')
    df.columns = new_columns
    df = df[2:]
    df = df.reset_index(drop=True)
    df = df.drop(['Image'], axis=1)

    df.to_csv(df_path)
    # df_drop = df[[col for col in df.columns if col.endswith('_x') or col.endswith('_y')]]

In [90]:
def fix_time_df(df_path):
    df = pd.read_csv(df_path, header=None, names=['DateTime', 'Seconds'])
    df = df.replace(r'[()]', '', regex=True)
    df.to_csv(df_path)

In [91]:
folders = [os.path.join(dst_dir, d) for d in os.listdir(dst_dir) if os.path.isdir(os.path.join(dst_dir, d))]
for folder in folders:
    for filename in os.listdir(folder):
        if "VoidTiming" in filename:
            fix_void_timing(os.path.join(folder,filename))
        elif "Pose_Data" in filename:
            fix_pose_data(os.path.join(folder,filename))
        elif "Side_view" in filename:
            fix_time_df(os.path.join(folder,filename))
        elif "Bottom_camera" in filename:
            fix_time_df(os.path.join(folder,filename))

/tmp/ipykernel_17913/2879870463.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path)
/tmp/ipykernel_17913/2879870463.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path)
/tmp/ipykernel_17913/2879870463.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_cs